In [174]:
import os
import pandas as pd
import pandas_read_xml as pdx
import openpyxl

# auto-indent: alt + shift + f

In [3]:
os.chdir('..')
os.getcwd() # output should be languagelens, not scripts

'/Users/nhuquynh/Documents/repos/languagelens'

In [4]:
pd.set_option('display.max_colwidth', None)

In [17]:
# Parse xml file
df = pdx.read_xml('data/WKR_220001_221643.xml', ['artikel-liste'])

In [18]:
# Flatten data
df = pdx.flatten(df)

In [179]:
# Separate text and meta data
df2 = df['artikel'].apply(pd.Series)

# Isolate meta data
df_meta = df2['metadaten'].apply(pd.Series)
meta_2 = df_meta['quelle'].apply(pd.Series)
# df_meta['artikel-id']

# Isolate text data
df_text = df2['inhalt'].apply(pd.Series)
df_text_info = df_text['titel-liste'].apply(pd.Series)
text = df_text['text'].apply(pd.Series)

# Combine into one file
meta_2['article_id'] = df_meta['artikel-id']
df_merge = meta_2.reset_index(drop=True).join(df_text_info)

# Rearrange columns
df_final = df_merge[['article_id', 'quelle-id', 'name', 'jahrgang', 'datum',
                     'seite-start', 'ressort', 'dachzeile', 'titel', 'untertitel']]

df_final.rename(columns={'quelle-id': 'newspaper_id',
                         'name': 'newspaper', 
                         'jahrgang': 'year', 
                         'seite-start': 'page_start', 
                         'titel': 'title', 
                         'untertitel': 'subtitle', 
                         'datum': 'date'}, inplace=True)

# Transform date to datetime format
df_final['date'] =  pd.to_datetime(df_final['date'], format='%Y%m%d').dt.strftime('%d.%m.%Y')


In [181]:
# Remove genios style and None from text column
text1 = []
for i in text.p:
    text1.append(([para.replace('</genios:style>', '').replace('<genios:style type="italic">',
          '').replace('<genios:style type="bold">', '').replace('<?agev_?>', '') for para in i if
           para is not None and
           para != '']))

# Add text column to final dataset
df_final['text'] = text1

In [182]:
# Save as excel
df_final.to_excel("output/format_example.xlsx")  